In [42]:
import pandas as pd
import requests
import time

In [43]:
# API key de Genderize.io
API_KEY = "e7dd0ce8ff5f451066328577c9d847e9"

In [44]:
# Cargar el dataset
df = pd.read_csv("datasets/fundadores_filtrados_etapa_temprana.csv", sep=',') 


In [45]:
df.head()

,founder_name,company_name,last_funding_type,gender
0,Annanay Kapila,QFEX,pre_seed,female
1,Joshua Wharton,QFEX,pre_seed,male
2,Georges Casassovici,Novoflow,pre_seed,male
3,Nikhil Tiwari,Frekil,pre_seed,male
4,Shivesh Gupta,Frekil,pre_seed,male


In [46]:
df.count()

founder_name         8404
company_name         8404
last_funding_type    8404
gender                 99
dtype: int64

In [47]:
# Solo procesar las filas donde 'gender' está vacío o es nulo
df["gender"] = df["gender"].fillna("")  # Asegura que existe la columna
df["first_name"] = df["founder_name"].fillna("").apply(lambda x: x.split()[0] if x else "")
df_to_process = df[df["gender"] == ""]


In [48]:
df_to_process.count()

founder_name         8305
company_name         8305
last_funding_type    8305
gender               8305
first_name           8305
dtype: int64

In [49]:
# Obtener nombres únicos a procesar
unique_names = df_to_process["first_name"].dropna().unique()
unique_names = [name for name in unique_names if name.strip()]

# Diccionario para almacenar resultados
gender_dict = {}

In [50]:
len(unique_names)

3347

In [51]:
print(unique_names)

['Sachitt', 'Josh', 'Mason', 'Alok', 'Nuha', 'Abhi', 'Graham', 'Zyad', 'Ananth', 'Connor', 'Sankeerth', 'Akber', 'Kayla', 'Saner', 'Ben', 'Ross', 'Garreth', 'Jason', 'Kenneth', 'Naijide', 'Caroline', 'Mats', 'Matthew', 'Chris', 'Daniel', 'Emmett', 'Sam', 'Sanio', 'Edward', 'Rajsekhar', 'Albert', 'Shardool', 'Anupom', 'Lander', 'Manu', 'Samuel', 'Alexandre', 'Chrisjan', 'Abhinav', 'Thomas', 'Boris', 'Salifyanji', 'Alex', 'Dimitrios', 'George', 'Filip', 'Jonathan', 'Philip', 'Jarren', 'Joonghyun', 'Oliver', 'Matan', 'Tomer', 'David', 'Shaun', 'Glenn', 'John', 'Oli', 'Rhim', 'Shubham', 'Vardhan', 'Gunwoo', 'Yong', 'Augustin', 'Baptiste', 'Papa', 'Shaashwat', 'Raphael', 'Tyler', 'Will', 'Dean', 'Etienne', 'Nathan', 'Maxime', 'Taieb', 'Leopold', 'Marvin', 'Brandon', 'Joel', 'Alix', 'Clément', 'Joseph', 'Sasha', 'Ciarán', 'Ismail', 'Ivan', 'Jack', 'Jacob', 'Geoffrey', 'Joachim', 'Amay', 'Ray', 'Nitish', 'Rishi', 'Bolu', 'Noah', 'Caelean', 'Evan', 'Bernardo', 'Gabriel', 'Gustavo', 'João', 'Ch

In [52]:
# 🔁 Función para procesar nombres en lotes de 10
def infer_gender_in_batches(names, api_key):
    results = {}
    for i in range(0, len(names), 10):
        batch = names[i:i + 10]
        names_param = "&".join([f"name[{j}]={n}" for j, n in enumerate(batch)])
        url = f"https://api.genderize.io/?{names_param}&apikey={api_key}"
        try:
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                for item in data:
                    name = item["name"]
                    gender = item.get("gender")
                    print(f"🔎 Procesado: {name} → {gender}")
                    results[name] = gender
            else:
                print(f"⚠️ Error en batch {batch}: {response.status_code}")
        except Exception as e:
            print(f"❌ Excepción en batch {batch}: {e}")
        time.sleep(1)  # evitar rate limit
    return results


In [53]:
# Procesar e inferir géneros
gender_dict = infer_gender_in_batches(unique_names, API_KEY)

🔎 Procesado: Sachitt → None
🔎 Procesado: Josh → male
🔎 Procesado: Mason → male
🔎 Procesado: Alok → male
🔎 Procesado: Nuha → female
🔎 Procesado: Abhi → male
🔎 Procesado: Graham → male
🔎 Procesado: Zyad → male
🔎 Procesado: Ananth → male
🔎 Procesado: Connor → male
🔎 Procesado: Sankeerth → male
🔎 Procesado: Akber → male
🔎 Procesado: Kayla → female
🔎 Procesado: Saner → male
🔎 Procesado: Ben → male
🔎 Procesado: Ross → male
🔎 Procesado: Garreth → male
🔎 Procesado: Jason → male
🔎 Procesado: Kenneth → male
🔎 Procesado: Naijide → None
🔎 Procesado: Caroline → female
🔎 Procesado: Mats → male
🔎 Procesado: Matthew → male
🔎 Procesado: Chris → male
🔎 Procesado: Daniel → male
🔎 Procesado: Emmett → male
🔎 Procesado: Sam → male
🔎 Procesado: Sanio → male
🔎 Procesado: Edward → male
🔎 Procesado: Rajsekhar → male
🔎 Procesado: Albert → male
🔎 Procesado: Shardool → male
🔎 Procesado: Anupom → male
🔎 Procesado: Lander → male
🔎 Procesado: Manu → male
🔎 Procesado: Samuel → male
🔎 Procesado: Alexandre → male
🔎 Proc

In [54]:
# Asignar resultados al DataFrame
df.loc[df["gender"] == "", "gender"] = df.loc[df["gender"] == "", "first_name"].map(gender_dict)

# Limpiar columna auxiliar
df.drop(columns=["first_name"], inplace=True)

# Guardar el resultado
df.to_csv("datasets/fundadores_etapa_temprana_con_genero.csv", index=False)
print("✅ Proceso completado. Archivo guardado como 'dataset_con_genero.csv'")

✅ Proceso completado. Archivo guardado como 'dataset_con_genero.csv'
